In [62]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=2)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
from eden.converter.molecule import obabel
import networkx as nx
import pybel
import requests
import os.path
from itertools import tee
from numpy.random import randint
from numpy.random import uniform
from eden.graph import Vectorizer
from sklearn.linear_model import SGDClassifier
import datetime, time
from eden.util import random_bipartition_iter
from eden.model import ActiveLearningBinaryClassificationModel

In [50]:
def make_iterable(filename, file_format):
    if file_format == 'sdf':
        with open(filename) as f:
            s = ''
            for line in f:
                if line.strip() != '$$$$':
                    s = s + line
                else:
                    return_value = s + line
                    s = ''
                    yield return_value
    elif file_format == 'smi':
        with open(filename) as f:
            for line in f:
                yield line

This is where the data sets are defined:

In [51]:
AID=1905
#AID=2401
DATA_DIR = '/home/liconj/proj/thesis/EDeN/examples/model_comparison/data'
active_fname=DATA_DIR + '/AID%s_active.sdf'%AID
inactive_fname=DATA_DIR + '/AID%s_inactive.sdf'%AID

In [52]:
active_fname

'/home/liconj/proj/thesis/EDeN/examples/model_comparison/data/AID1905_active.sdf'

Functions for training and testing the model

In [66]:
model_fname = DATA_DIR + '/AID%s.model3d'%AID
model_type = "3d"
n_conf = 10
n_iter = 10
active_set_size = 5
n_active_learning_iterations = 0
threshold = 1
train_test_split = 0.8
verbosity = 1


pre_processor_parameters={'k':randint(1, 10,size=n_iter),
                          'threshold':randint(3, 10, size=n_iter),
                          'model_type':["default"],
                          'n_conf':[n_conf]}

def pre_processor(data, model_type="3d", **kwargs):
    # model_type = kwargs.get('mode', 'default')
    if model_type == "default":
        iterable = obabel.obabel_to_eden(data, **kwargs)
    elif model_type == "3d":
        iterable = obabel.obabel_to_eden3d(data, **kwargs)
    return iterable



vectorizer = Vectorizer()
estimator = SGDClassifier(class_weight='auto', shuffle=True)

# Make predictive model
model = ActiveLearningBinaryClassificationModel(pre_processor,
                                                estimator=estimator,
                                                vectorizer=vectorizer,
                                                n_jobs=2,
                                                n_blocks = 10,
                                                fit_vectorizer=True)

In [67]:
########
# Create iterables from files
########

iterable_pos = make_iterable('AID720577_active.sdf', 'sdf')
iterable_neg = make_iterable('AID720577_inactive.sdf', 'sdf')
iterable_pos, iterable_pos_ = tee(iterable_pos)
iterable_neg, iterable_neg_ = tee(iterable_neg)

start = time.time()
print('# positives: %d  # negatives: %d (%.1f sec %s)'%(sum(1 for x in iterable_pos_), sum(1 for x in iterable_neg_), time.time() - start, str(datetime.timedelta(seconds=(time.time() - start)))))


iterable_pos, iterable_pos_ = tee(iterable_pos)
iterable_neg, iterable_neg_ = tee(iterable_neg)

# Split train/test
iterable_pos_train, iterable_pos_test = random_bipartition_iter(iterable_pos, relative_size=train_test_split)
iterable_neg_train, iterable_neg_test = random_bipartition_iter(iterable_neg, relative_size=train_test_split)

# positives: 80  # negatives: 146 (0.2 sec 0:00:00.231483)


In [ ]:
%%time
# Optimize hyperparameters and fit model
# Since this model is fitted much more slowly, use a single vectorizer
#vectorizer_parameters={'complexity':[2,3,4],
#                       'discretization_size':randint(2, 3,size=n_iter),
#                       'discretization_dimension':randint(2, 3,size=n_iter)}

vectorizer_parameters={'complexity':[1]}


estimator_parameters={'n_iter':randint(5, 100, size=n_iter),
                      'penalty':['l1','l2','elasticnet'],
                      'l1_ratio':uniform(0.1,0.9, size=n_iter),
                      'loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
                      'power_t':uniform(0.1, size=n_iter),
                      'alpha': [10**x for x in range(-8,-2)],
                      'eta0': [10**x for x in range(-4,-1)],
                      'learning_rate': ["invscaling", "constant", "optimal"]}

model.optimize(iterable_pos_train, iterable_neg_train,
               model_name=model_fname,
               n_active_learning_iterations=0,
               size_positive=-1,
               size_negative=active_set_size,
               n_iter=n_iter, cv=3,
               pre_processor_parameters=pre_processor_parameters,
               vectorizer_parameters=vectorizer_parameters,
               estimator_parameters=estimator_parameters)



	Parameters range:

Pre_processor:
         k: [6 7 7 5 8 5 6 5 7 3]
model_type: ['default']
    n_conf: [10]
 threshold: [4 8 9 3 5 9 8 7 3 8]

Vectorizer:
complexity: [1]

Estimator:
     alpha: [1e-08, 1e-07, 1e-06, 1e-05, 0.0001, 0.001]
      eta0: [0.0001, 0.001, 0.01]
  l1_ratio: [ 0.84636909  0.41693706  0.60857139  0.50517529  0.58082707  0.36103254
  0.85310045  0.89238478  0.72720053  0.1261606 ]
learning_rate: ['invscaling', 'constant', 'optimal']
      loss: ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron']
    n_iter: [12 64 91 15 42 54 28 54 40 98]
   penalty: ['l1', 'l2', 'elasticnet']
   power_t: [ 0.1019144   0.77685118  0.15121337  0.73396505  0.18820267  0.94383539
  0.22610404  0.29338202  0.9318943   0.33687789]

Failed iteration: 1/10 (at 0.6 sec; 0:00:00.631541)
Inappropriate argument type.
coercing to Unicode: need string or buffer, list found
Failed with the following setting:

	Model parameters:

Pre_processor:
         k: 6
model_type: defau

In [48]:
%%time
# Estimate predictive performance
model.estimate( iterable_pos_test, iterable_neg_test )

TypeError: 'NoneType' object is not callable

In [ ]:
def test_obabel_model(fname, model_type = "default", model_fname=None):
    from eden.model import ActiveLearningBinaryClassificationModel

    model = ActiveLearningBinaryClassificationModel()
    model.load(model_fname)

    #create iterable from files
    from eden.converter.molecule import obabel
    if model_type == "default":
        iterable=obabel.obabel_to_eden(fname)
    elif model_type == "3d":
        iterable=obabel.obabel_to_eden3d(fname)

    predictions= model.decision_function( iterable )

    return predictions